In [47]:
import numpy as np
import pandas as pd
import datetime as dt
import spacy
nlp = spacy.load('en_core_web_sm')
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from wordcloud import WordCloud
import matplotlib as plt

In [2]:
tik_tok_df = pd.read_csv('tiktok_app_reviews.csv')

/Users/alexstake/opt/anaconda3/envs/metis/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
tik_tok_df =tik_tok_df.drop(columns='review_id')

In [4]:
tik_tok_df.head()

,review_text,upvotes,app_version,posted_date
0,It is an excellent solution to finding videos ...,402,21.2.5,2021-09-20 07:50:23
1,Great app works really well so many features s...,67,21.2.5,2021-09-19 23:10:47
2,I down loaded app but didnt work . i love this...,24,21.2.5,2021-09-19 08:59:34
3,IT A GREAT APP AND FUN TO USE! HOWEVER ARBITRA...,18,21.2.5,2021-09-19 16:34:43
4,Currently it's working good for me except a fe...,232,21.1.41,2021-09-14 06:35:02


In [5]:
tik_tok_df['date'] = pd.to_datetime(tik_tok_df['posted_date'], infer_datetime_format= True).dt.date

In [6]:
split_date= dt.date(2020,1,1)

In [7]:
tik_tok_20_21 = tik_tok_df.loc[tik_tok_df['date']>split_date]

In [8]:
tik_tok_20_21.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1937876 entries, 0 to 3646475
Data columns (total 5 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   review_text  object
 1   upvotes      int64 
 2   app_version  object
 3   posted_date  object
 4   date         object
dtypes: int64(1), object(4)
memory usage: 88.7+ MB


In [9]:
tik_tok_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3646476 entries, 0 to 3646475
Data columns (total 5 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   review_text  object
 1   upvotes      int64 
 2   app_version  object
 3   posted_date  object
 4   date         object
dtypes: int64(1), object(4)
memory usage: 139.1+ MB


In [10]:
tik_tok_20_21.review_text.isnull().sum()

16

Only 16 null reviews out of 1.9 million, so can probably drop them without too much impact

In [11]:
tik_tok_20_21.loc[tik_tok_20_21.review_text.isnull()]

,review_text,upvotes,app_version,posted_date,date
1713596,NaN,0,19.5.43,2021-06-16 11:39:27,2021-06-16
1724879,NaN,0,19.5.4,2021-05-19 03:56:05,2021-05-19
1729379,NaN,0,19.3.4,2021-05-25 23:51:17,2021-05-25
1762648,NaN,0,18.8.41,2021-04-09 05:12:47,2021-04-09
1782371,NaN,0,18.6.6,2021-03-04 19:01:34,2021-03-04
1824753,NaN,0,18.3.6,2021-01-31 07:27:14,2021-01-31
2079417,NaN,0,15.9.6,2020-05-11 14:45:00,2020-05-11
2117121,NaN,0,15.7.5,2020-04-20 08:48:29,2020-04-20
2144000,NaN,0,15.5.42,2020-05-04 01:45:30,2020-05-04
2212880,NaN,0,15.2.4,2020-03-15 04:03:22,2020-03-15


In [12]:
no_null_reviews_df = tik_tok_20_21.dropna(subset=['review_text'])

In [13]:
no_null_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1937860 entries, 0 to 3646475
Data columns (total 5 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   review_text  object
 1   upvotes      int64 
 2   app_version  object
 3   posted_date  object
 4   date         object
dtypes: int64(1), object(4)
memory usage: 88.7+ MB


In [14]:
no_null_reviews_df['spacy_cleaned'] = list(nlp.pipe(no_null_reviews_df.review_text))

/var/folders/ct/v_q82qz551vg3s2wptrzz4cw0000gp/T/ipykernel_4739/2548810918.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_null_reviews_df['spacy_cleaned'] = list(nlp.pipe(no_null_reviews_df.review_text))


In [15]:
no_null_reviews_df.head()

,review_text,upvotes,app_version,posted_date,date,spacy_cleaned
0,It is an excellent solution to finding videos ...,402,21.2.5,2021-09-20 07:50:23,2021-09-20,"(It, is, an, excellent, solution, to, finding,..."
1,Great app works really well so many features s...,67,21.2.5,2021-09-19 23:10:47,2021-09-19,"(Great, app, works, really, well, so, many, fe..."
2,I down loaded app but didnt work . i love this...,24,21.2.5,2021-09-19 08:59:34,2021-09-19,"(I, down, loaded, app, but, did, nt, work, ., ..."
3,IT A GREAT APP AND FUN TO USE! HOWEVER ARBITRA...,18,21.2.5,2021-09-19 16:34:43,2021-09-19,"(IT, A, GREAT, APP, AND, FUN, TO, USE, !, HOWE..."
4,Currently it's working good for me except a fe...,232,21.1.41,2021-09-14 06:35:02,2021-09-14,"(Currently, it, 's, working, good, for, me, ex..."


In [16]:
no_null_reviews_df.spacy_cleaned[29]

No glitches or hick ups as of yet, rather enjoying using the app 👍 Update, tried to log into my account via phone, says server error but our internet is fine, tried signing in with my facebook account, still server error, is the site under maintenance by any chance? Or will I need to uninstall then re-install?

Looks like the parser worked, emojis are still in there but I believe they can stay since they can sometimes show patterns that are positive

In [17]:
docs_clean = [[w.lemma_.lower() for w in doc if (not w.is_stop and not w.is_punct and not w.like_num) \
               or (w.lemma_=='not')] for doc in no_null_reviews_df['spacy_cleaned']]
no_null_reviews_df['docs_clean'] = docs_clean

/var/folders/ct/v_q82qz551vg3s2wptrzz4cw0000gp/T/ipykernel_4739/3201625770.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_null_reviews_df['docs_clean'] = docs_clean


In [18]:
no_null_reviews_df.head()

,review_text,upvotes,app_version,posted_date,date,spacy_cleaned,docs_clean
0,It is an excellent solution to finding videos ...,402,21.2.5,2021-09-20 07:50:23,2021-09-20,"(It, is, an, excellent, solution, to, finding,...","[excellent, solution, find, video, follow, peo..."
1,Great app works really well so many features s...,67,21.2.5,2021-09-19 23:10:47,2021-09-19,"(Great, app, works, really, well, so, many, fe...","[great, app, work, feature, new, feature, lot,..."
2,I down loaded app but didnt work . i love this...,24,21.2.5,2021-09-19 08:59:34,2021-09-19,"(I, down, loaded, app, but, did, nt, work, ., ...","[load, app, nt, work, love, app, maybe, fix, b..."
3,IT A GREAT APP AND FUN TO USE! HOWEVER ARBITRA...,18,21.2.5,2021-09-19 16:34:43,2021-09-19,"(IT, A, GREAT, APP, AND, FUN, TO, USE, !, HOWE...","[great, app, fun, use, arbitrary, algorithms, ..."
4,Currently it's working good for me except a fe...,232,21.1.41,2021-09-14 06:35:02,2021-09-14,"(Currently, it, 's, working, good, for, me, ex...","[currently, work, good, clip, video, work, hou..."


In [24]:
docs_list_clean = [' '.join(doc) for doc in docs_clean]
docs_list_clean[:4]

['excellent solution find video follow people similar interest excellent solution rediscover environment society give respect environment society public view private view',
 'great app work feature new feature lot popular trend thing app like find friend post video like duet share yes sponsor tik tok scam app pretty content foryou page wo load great app continue improve yes stars',
 'load app nt work love app maybe fix bug easy load well like sign facebook google work good phone tho ipad not work fix',
 'great app fun use arbitrary algorithms blocking banning takedowns etc alleged report reporter able report evidence evidence acquire action evidence require uncorroborated report[s wreck persons rep account -if inclusive environment prove needs far well customer service online phone u know phone app great invent people talk']

In [41]:
cv = CountVectorizer(stop_words='english', max_df=.9, strip_accents='unicode')
tfid =TfidfVectorizer(stop_words='english', max_df=.9, strip_accents='unicode')

In [42]:
X1 = cv.fit_transform(docs_list_clean)
X2 = tfid.fit_transform(docs_list_clean)

In [54]:
print(X1)
print(X2)

  (0, 65837)	2
  (0, 187613)	2
  (0, 213109)	1
  (0, 73137)	1
  (0, 153847)	1
  (0, 184952)	1
  (0, 167182)	1
  (0, 63947)	2
  (0, 187133)	2
  (0, 168624)	1
  (0, 160728)	1
  (0, 213494)	2
  (0, 159656)	1
  (1, 213109)	1
  (1, 82260)	2
  (1, 20782)	4
  (1, 219744)	1
  (1, 70646)	2
  (1, 142281)	1
  (1, 122053)	1
  (1, 157635)	1
  (1, 203915)	1
  (1, 199586)	1
  (1, 119770)	2
  (1, 74738)	1
  :	:
  (1937834, 192961)	1
  (1937835, 128273)	1
  (1937836, 192961)	1
  (1937837, 78487)	1
  (1937838, 142649)	1
  (1937839, 200898)	1
  (1937840, 177179)	1
  (1937841, 198574)	1
  (1937842, 75696)	1
  (1937843, 219985)	1
  (1937844, 27141)	1
  (1937845, 223077)	1
  (1937846, 128273)	1
  (1937847, 236088)	1
  (1937849, 68633)	1
  (1937850, 138163)	1
  (1937851, 22025)	1
  (1937852, 133642)	1
  (1937853, 80728)	1
  (1937854, 134907)	1
  (1937855, 219985)	1
  (1937856, 141467)	1
  (1937857, 141467)	1
  (1937858, 142987)	1
  (1937859, 142743)	1
  (0, 159656)	0.18769900290485866
  (0, 213494)	0.2528742

In [63]:
lsa = TruncatedSVD(n_components=10)
lsa.fit(X2)

TruncatedSVD(n_components=10)

In [61]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [64]:
display_topics(lsa, tfid.get_feature_names(), 25)


Topic  0
good, app, nice, love, tiktok, like, amazing, tik, tok, great, fun, video, entertainment, awesome, world, time, super, cool, funny, aap, enjoy, thing, use, experience, people

Topic  1
nice, app, love, tiktok, amazing, tik, tok, great, like, fun, awesome, video, cool, super, funny, aap, enjoy, wow, thank, people, use, make, entertaining, time, watch

Topic  2
love, app, tiktok, tok, tik, amazing, fun, great, like, video, awesome, cool, funny, make, watch, dance, people, day, super, thank, lot, friend, entertaining, life, use

Topic  3
app, amazing, great, like, fun, awesome, super, cool, enjoy, funny, entertainment, excellent, video, world, use, best, time, entertaining, interesting, wonderful, favorite, nyc, download, wow, osm

Topic  4
amazing, tik, tok, like, fun, video, tiktok, nice, good, thank, viral, funny, watch, cool, plz, account, people, super, follower, make, aap, day, lot, team, follow

Topic  5
amazing, love, nice, good, wow, aap, word, absolutely, tiktok, appli